# <center>Адаптивная Арифметика</center>
# <center>Adaptive precision floating-point arithmetic</center>

## Основные понятия

$ x = \pm \mathrm{significand} \times 2^{\mathrm{exp}}$, где $\mathrm{significand} = \mathrm{b.bb\dots b}$ — $p$ бит

**Разложение** (*англ. expansion*) $x = x_n + \dots + x_1 + x_0$, где каждое $x_i$ является компонентой числа $x$ и представляется в виде $\mathrm{p}$-битного числа с плавающей точкой.

$x$ и $y$ называются **неперекрывающимися** (*англ. nonoverlapping*), если номер наименьшего значимого бита числа $x$ (нумерация справа налево) больше, чем номер наибольшего значимого бита числа $y$, или наоборот (Если существуют такие целые числа $r$ и $s$, что $x = r\times2^s$ и $|y| < 2^s$, или $y = r\times2^s$ и $|x| < 2^s$). Ноль не перекрывается ни с одним другим числом.
Например, числа $1100$ и $-10.1$ не перекрываются, а $110$ и $10$ — перекрываются.

$x$ и $y$ называются **смежными** (*англ. adjacent*), если они перекрываются, если $x$ и $2y$ перекрываются, или $2x$ и $y$ перекрываются. Например, числа $1100$ и $11$ смежные, а $1000$ и $11$ — нет.

Все алгоритмы предполагают, что сложение, вычитание и умножение выполняются с **точным округлением** (*англ. exact rounding*). **Округление до ближайшего четного** (*англ. round-to-even*) — правило, при котором округление производится к ближайшему $p$-битному четному значению. **Округление к нулю** (*англ. round-toward-zero*) — правило, при котором округление производится к ближайшему $p$-битному значению, находящемуся между точным значением и нулем.

При анализе округления часто используют оператор $\mathrm{ulp}$ (*англ. unit in the last place*) — эффективную величину самого младшего $p$-ого бита. Он определяется для конкретного числа: $\mathrm{ulp}(x) = 2^{-p+1+\mathrm{exp}}$.

Также используют оператор $\mathrm{err}(a\odot b)$, который показывает ошибку округления результата выполнения операции $\odot$. Необходимо отметить, что $\mathrm{ulp}$ беззнаковая величина, а $\mathrm{err}$ всегда имеет знак. Для операций сложения, вычитания и умножения с точным округлением выполняется равенство:
$$ a\odot b = a\cdot b  + \mathrm{err}(a\odot b), где   \left | \mathrm{err}(a\odot b) \right | \leqslant \frac{1}{2}\mathrm{ulp}(a\odot b)$$

## Свойства двоичной арифметики

> **Лемма 1** Пусть $a \oplus b = a + b + \mathrm{err} (a \oplus b) $.
Тогда ошибка округления $\mathrm{err}(a \oplus b)$ не больше чем $\left|a\right|$ или $\left|b\right|$. (Аналогично для вычитания)

> **Следствие 2** Ошибка округления $\mathrm{err}(a \oplus b)$ может быть представлена в $p$ битах

> **Лемма 3** Пусть $a \odot b = a \cdot b + \mathrm{err} (a \odot b) $.
Тогда 

>a) Если $\left|\mathrm{err} (a \odot b)\right| \geqslant 2^i$ для целого $i$, то $\left|a\cdot b\right| \geqslant 2^i(2^p+1)$

>б) Если $\left|\mathrm{err} (a \odot b)\right| > 2^i$ для целого $i$, то $\left|a\cdot b\right| > 2^i(2^{p+1}+1)$

<center><img src="img/lemma1.png"/></center>
<font size="2"><i><center>Рис. 1</center>Вертикальные линии обозначают 4-битные значения с плавающей точкой. Ошибка округления — расстояние между $a+b$ и $a \oplus b$. Лемма 1 утверждает, что ошибка не может превышать $\left|b\right|$. Лемма 3б) утверждает, что если $\left|(a+b)\right| \leqslant 2^i(2^{p+1}+1)$ (для $i=-2$ и $p=4$ величина $a+b$ попадает на рисунке в выделенный регион), то ошибка не превышает $2^i$. Эта лемма полезна, когда значение близко к степени 2.</i></font>

> **Лемма 4** Пусть $\left|a + b \right| \leqslant \left|a\right|$ и $\left|a + b \right| \leqslant \left|b\right|$. Тогда $a \oplus b = a + b$. (Аналогично для вычитания)

> **Лемма 5** Пусть $b \in \left [ \frac{a}{2}, 2a \right ]$. Тогда $a \ominus b = a - b$.

<center><img src="img/lemma5.png"/></center>
<font size="2"><i><center>Рис. 2<br/>Примеры к лемме 5</center></i></font>

## Простое суммирование

> **Теорема 6 (Dekker)** Пусть $a$ и $b$ — $p$-битные числа, причем $\left|a\right| \geqslant \left|b\right|$. Тогда следующий алгоритм вернет неперекрывающееся разложение $x + y$ такое, что $a + b = x + y$, где $x$ - приближение суммы $a + b$, а $y$ представляет собой ошибку округления при вычислении $x$.

<div style="padding-left:100px">
Fast-Two-Sum (a, b)<br/>
1. $x \Leftarrow a \oplus b$<br/>
2. $b_{virtual} \Leftarrow x \ominus a$<br/>
3. $y \Leftarrow b \ominus b_{virtual}$<br/>
4. return $(x, y)$<br/>
</div>
<br/>
<div style="padding-left:100px">
Fast-Two-Diff (a, b)<br/>
1. $x \Leftarrow a \ominus b$<br/>
2. $b_{virtual} \Leftarrow a \ominus x$<br/>
3. $y \Leftarrow b_{virtual} \ominus b$<br/>
4. return $(x, y)$<br/>
</div>

<center><img src="img/fts1.png"/></center>
<center><br><font size="2"><i>Рис. 3<br/> Демонстрация алгоритма Fast-Two-Sum, когда $a$ и $b$ имеют один знак. $111100 + 1001 = 1001000 + -11$</i></font></center>

<center><img src="img/fts2.png"/></center>
<center><br><font size="2"><i>Рис. 4<br/> Демонстрация алгоритма Fast-Two-Sum, когда $a$ и $b$ имеют разный знак и $\left|b\right| > \frac{\left|a\right|}{2}$. $10010 + -1011 = 111 + 0$</i></font></center>

> **Теорема 7 (Knuth)** Пусть $a$ и $b$ — $p$-битные числа, причем $p \geqslant 3$. Тогда следующий алгоритм вернет неперекрывающееся разложение $x + y$ такое, что $a + b = x + y$, где $x$ - приближение суммы $a + b$, а $y$ представляет собой ошибку округления при вычислении $x$.

<div style="padding-left:100px">
Two-Sum (a, b)<br/>
1. $x \Leftarrow a \oplus b$<br/>
2. $b_{virtual} \Leftarrow x \ominus a$<br/>
3. $a_{virtual} \Leftarrow x \ominus b_{virtual}$<br/>
4. $b_{roundoff}\Leftarrow b \ominus b_{virtual}$<br/>
5. $a_{roundoff}\Leftarrow a \ominus a_{virtual}$<br/>
6. $y \Leftarrow a_{roundoff} \oplus b_{roundoff}$<br/>
7. return $(x, y)$<br/>
</div>
<br\>
<div style="padding-left:100px">
Two-Diff (a, b)<br/>
1. $x \Leftarrow a \ominus b$<br/>
2. $b_{virtual} \Leftarrow a \ominus x$<br/>
3. $a_{virtual} \Leftarrow x \oplus b_{virtual}$<br/>
4. $b_{roundoff}\Leftarrow b_{virtual} \ominus b$<br/>
5. $a_{roundoff}\Leftarrow a \ominus a_{virtual}$<br/>
6. $y \Leftarrow a_{roundoff} \oplus b_{roundoff}$<br/>
7. return $(x, y)$<br/>
    </div>

<center><img src="img/ts1.png"/></center>
<center><br><font size="2"><i>Рис. 5<br/> Демонстрация алгоритма Two-Sum, когда $\left|a\right| < \left|b\right|$ и $\left|a\right| \leqslant \left|x\right|$. $11.11 + 1101 = 10000 + 0.11$</i></font></center>

> **Следствие 8** Пусть $x$ и $y$ значения, которые вернули функции Fast-Two-Sum или Two-Sum.
Тогда 

>a) Если $\left|y\right| \geqslant 2^i$ для целого $i$, то $\left|x + y\right| \geqslant 2^i(2^p+1)$

>б) Если $\left|y\right| > 2^i$ для целого $i$, то $\left|x + y\right| > 2^i(2^{p+1}+1)$

> **Следствие 9** Пусть $x$ и $y$ значения, которые вернули функции Fast-Two-Sum или Two-Sum.
Тогда, при использовании округления до ближайшего четного, $x$ и $y$ не смежные.

## Суммирование разложений

> **Теорема 10** Пусть $e = \sum^{m}_{i=1}e_i$ - неперекрывающееся $m$-компонентное  разложение; $b$ — $p$-битное число, где $p \geqslant 3$. Предполагается, что $e_1, e_2, \dots, e_m$ отсортированы в *возрастающем* по величине порядке, за исключением того, что любая компонента может равняться $0$. Тогда следующий алгоритм вернет неперекрывающееся разложение $h = \sum^{m + 1}_{i=1}h_i = e + b$, где компоненты $h_1, h_2, \dots, h_{m+1}$ также отсортированы в возрастающем порядке, за исключением того, что любая компонента может равняться $0$. Если разложение $e$ несмежное и используется округление к ближайшему четному, то разложение $h$ тоже несмежное.


<div style="padding-left:100px">
Grow-Expansion(e, b)<br/>
1. $Q_0 \Leftarrow b$<br/>
2. for $i \Leftarrow 1$ to $m$<br/>
3. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$(Q_i, h_i) \Leftarrow$ Two-Sum$(Q_{i-1}, e_i)$<br/>
4. $h_{m+1} \Leftarrow Q_m$<br/>
5. return $h$<br/>
</div>

<center><img src="img/ge.png"/></center>
<center><br><font size="2"><i>Рис. 6<br/> Алгоритм Grow-Expansion. Разложения $e$ и $h$ имеют большие компоненты слева. Большее значение операции Two-Sum выходит влево, а меньшее вниз.</i></font></center>

> **Следствие 11** Каждая из первых $m$ компонет разложения $h$ не больше, чем соответствующая компонента разложения $e$. Кроме того $\left|h_1\right| \leqslant \left|b\right|$.

> **Теорема 12** Пусть $e = \sum^{m}_{i=1}e_i$ и $f = \sum^{n}_{i=1}f_i$ - неперекрывающиеся $m$-компонентное и $n$-компонентное $p$-битные разложения, где $p \geqslant 3$. Предполагается, что компоненты $e$ и $f$ отсортированы в *возрастающем* по величине порядке, за исключением того, что любая компонента может равняться $0$. Тогда следующий алгоритм вернет неперекрывающееся разложение $h = \sum^{m + n}_{i=1}h_i = e + f$, где компоненты $h_1, h_2, \dots, h_{m+1}$ также отсортированы в возрастающем порядке, за исключением того, что любая компонента может равняться $0$. Если разложения $e$ и $f$ несмежные и используется округление к ближайшему четному, то разложение $h$ тоже несмежное.


<div style="padding-left:100px">
Expansion-Sum(e, f)<br/>
1. $h \Leftarrow e$<br/>
2. for $i \Leftarrow 1$ to $n$<br/>
3. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\langle h_i, h_{i+1}, \dots, h_{i+m}\rangle \Leftarrow$ Grow-Expansion$(\langle h_i, h_{i+1}, \dots, h_{i+m - 1}\rangle, f_i)$<br/>
4. return $h$<br/>
</div>

<center><img src="img/es.png"/></center>
<center><br><font size="2"><i>Рис. 7<br/>Алгоритм Expansion-Sum</i></font></center>

Разложение называется **строго неперекрывающимся**, если его компоненты попарно не перекрываются, ни одна компонента не смежна никаким двум другим, а также любая пара является степенями двойки.

> **Теорема 13** Пусть $e = \sum^{m}_{i=1}e_i$ и $f = \sum^{n}_{i=1}f_i$ - строго неперекрывающиеся $m$-компонентное и $n$-компонентное $p$-битные разложения, где $p \geqslant 4$. Предполагается, что компоненты $e$ и $f$ отсортированы в *возрастающем* по величине порядке, за исключением того, что любая компонента может равняться $0$. Тогда, на машинах с правилом округления к ближайшему четному, следующий алгоритм вернет строго неперекрывающееся разложение $h = \sum^{m + n}_{i=1}h_i = e + f$, где компоненты $h_1, h_2, \dots, h_{m+1}$ также отсортированы в возрастающем порядке, за исключением того, что любая компонента может равняться $0$.


<div style="padding-left:100px">
Fast-Expansion-Sum(e, f)<br/>
1. Объединить последовательности $e$ и $f$ в одну неубывающую последовательность $g$, возможно с нулевыми компонентами<br/>
2. $(Q_2, h_1) \Leftarrow $ Fast-Two-Sum$(g_2, g_1)$<br/>
3. for $i \Leftarrow 3$ to $m + n$<br/>
4. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$(Q_i, h_{i-1}) \Leftarrow$ Two-Sum$(Q_{i-1}, g_i)$<br/>
5. $h_{m+n} \Leftarrow Q_{m+n}$<br/>
6. return $h$<br/>
</div>

<center><img src="img/fes.png"/></center>
<center><br><font size="2"><i>Рис. 8<br/>Алгоритм Fast-Expansion-Sum</i></font></center>

> **Лемма 14 (Q invariant)** В конце каждой итерации цикла $for$ сохраняется инвариант $Q_i + \sum_{j=1}^{i-1}h_j = \sum_{j=1}^ig_j$. Это означает, что после исполнения строки 5, $\sum_{j=1}^{m+n}h_j = \sum_{j=1}^{m+n}g_j$, и алгоритм выдает корректную сумму.

> **Лемма 15** Пусть $\hat{g} = \sum_{j=1}^{k}\hat{g}_j$ — последовательность, сформированная из двух строго неперекрывающихся разложений. Предположим что $\hat{g}_k$ — наибольшая компонента и имеет ненулевой бит величины $2^i$ или меньше для некоторого $i$. Тогда $\left|\sum_{j=1}^{k}\hat{g}_j\right| < 2^i(2^{p+1}-1)$ и $\left|\sum_{j=1}^{k-1}\hat{g}_j\right| < 2^i(2^p)$.

> **Лемма 16** Разложение $h$, полученное из алгоритма Fast-Expansion-Sum, является неперекрывающимся, и его компоненты расположены по возрастанию величины (исключая нули).

<center><img src="img/fes2.png"/></center>
<center><br><font size="2"><i>Рис. 9<br/>4-битовый пример алгоритма Fast-Expansion-Sum. $11110.1 + 10011110.1 = (11110 + 0.1) + (10000000 + 11110 + 0.1) = 11000000 + 0 + -10 + -1 + 0$</i></font></center>

## Простое умножение

> **Теорема 17** Пусть $a$ — $p$-битное число с плавающей точкой при $p \geqslant 3$. Выберем **точку разбиения** $s$, такую что $\frac{p}{2}\leqslant s \leqslant p-1$. Тогда следующий алгоритм вернет $(p-s)$-битное число $a_{hi}$ и $(s-1)$-битное число $a_{lo}$, такие что $\left|a_{hi}\right| \geqslant \left|a_{lo}\right|$ и $a = a_{hi} + a_{lo}$.

<div style="padding-left:100px">
Split(a, s)<br/>
1. $c \Leftarrow (2^s + 1)\otimes a$<br/>
2. $a_{big} \Leftarrow c\ominus a$<br/>
3. $a_{hi} \Leftarrow c\ominus a_{big}$<br/>
4. $a_{lo} \Leftarrow a\ominus a_{hi}$<br/>
5. return $(a_{hi}, a_{lo})$<br/>
</div>

<center><img src="img/split.png"/></center>
<center><br><font size="2"><i>Рис. 10<br/>Пример алгоритма Split, выполняющий разбиение $5$-битового числа на два $2$-битовых.$11101 = 10000 + -11$</i></font></center>

> **Теорема 18** Пусть $a$ и $b$ есть $p$-битные числа, причем $p \geqslant 6$. Тогда следующий алгоритм вернет неперекрывающееся разложение $x + y$ такое, что $a \times b = x + y$, где $x$ - приближение произведения $a \times b$, а $y$ представляет собой ошибку округления при вычислении $x$. Кроме того, если используется правило округления до ближайшего четного, то $x$ и $y$ несмежные.

<div style="padding-left:100px">
Two-Product(a, b)<br/>
1. $x \Leftarrow a\otimes b$<br/>
2. $(a_{hi}, a_{lo}) =$ Split$(a, \left \lceil \frac{p}{2} \right \rceil)$<br/>
3. $(b_{hi}, b_{lo}) =$ Split$(b, \left \lceil \frac{p}{2} \right \rceil)$<br/>
4. $err_1 \Leftarrow x\ominus (a_{hi}\otimes b_{hi})$<br/>
5. $err_2 \Leftarrow err_1\ominus (a_{lo}\otimes b_{hi})$<br/>
6. $err_3 \Leftarrow err_2\ominus (a_{hi}\otimes b_{lo})$<br/>
7. $y \Leftarrow (a_{lo}\otimes b_{lo}) \ominus err_3$<br/>
8. return $(x, y)$<br/>
</div>

<center><img src="img/tp.png"/></center>
<center><br><font size="2"><i>Рис. 11<br/>Пример алгоритма Two-Product в 6-битовой арифметике, где $a = b = 111011$, $a_{hi} = b_{hi} = 111000$, $a_{lo} = b_{lo} = 11$. Каждое промежуточное вычисление выражается в 6 битах. Итоговое выражение равно $110110\times2^6 + 11001$.</i></font></center>